### Main points

#### Group analysis across ROI

- Make average and sem of pRF size for fovea and periphery of controls / amblyo / aniso / strab / mixed
- Make average and sem of pRF size for fovea and periphery of controls / amblyo / aniso / strab / mixed
- Make difference of control with amblyo / aniso / strab / mixed of pRF size for fovea and periphery
- Make difference of control with / amblyo / aniso / strab / mixed of pRF CM for fovea and periphery
- Test significance of control vs. patient of pRF size for fovea and periphery
- Test significance of control vs. patient of pRF CM for fovea and periphery
- Test significance of fovea vs. periphery of pRF size
- Test significance of fovea vs. periphery of pRF CM
  

In [1]:
# General imports
import os
import sys
import json
import glob
import ipdb
import pandas as pd
import numpy as np
deb = ipdb.set_trace

In [2]:
# Inputs
data_dir = '/home/mszinte/disks/meso_S/data'
code_dir = '/home/mszinte/disks/meso_H/projects'
amblyo_dir = 'amblyo_prf'
control_dir = 'RetinoMaps'

# Personal import
sys.path.append("{}/{}/analysis_code/utils".format(code_dir, amblyo_dir))
from plot_utils import *

# load amblyo settings
with open('{}/{}/analysis_code/settings.json'.format(code_dir, amblyo_dir)) as f:
    json_s = f.read()
    amblyo_analysis_info = json.loads(json_s)
amblyo_subjects = amblyo_analysis_info['subjects_patient']
strab_subjects = amblyo_analysis_info['subjects_strab']
aniso_subjects = amblyo_analysis_info['subjects_aniso']
mixed_subjects = amblyo_analysis_info['subjects_mixed']
formats = amblyo_analysis_info['formats']
foveal_bound = amblyo_analysis_info['foveal_bound']
datasets = ['amblyo', 'control']

# load control settings
with open('{}/{}/analysis_code/settings.json'.format(code_dir, control_dir)) as f:
    json_s = f.read()
    control_analysis_info = json.loads(json_s)
control_subjects = control_analysis_info['subjects']


#### Indivudal analysis
Get for each subject, the weighted average pRF size and CM for foveal and peripheral pRF and save its type

In [25]:
# Amblyopic patients

for format_ in formats:
    print('Format: {}'.format(format_))
    
    for dataset in ['amblyo', 'control']:
        if dataset == 'amblyo':
            dataset_dir = amblyo_dir
            list_subjects = amblyo_subjects
        elif dataset == 'control':
            dataset_dir = control_dir
            list_subjects = control_subjects
        print('Dataset: {}'.format(dataset))
        
        for subject in list_subjects:
            # Load data thresholded
            tsv_dir = '{}/{}/derivatives/pp_data/{}/{}/prf/tsv'.format(
                data_dir, dataset_dir, subject, format_)
            tsv_data_th_fn = "{}/{}_prf_violins.tsv".format(tsv_dir, subject)
            data_th = pd.read_table(tsv_data_th_fn, sep="\t")
    
            # Define subject type
            if dataset == 'amblyo':
                if subject in strab_subjects: 
                    subject_type = 'patient'
                    amblyo_type = 'strab'
                elif subject in aniso_subjects: 
                    subject_type = 'patient'
                    amblyo_type = 'aniso'
                elif subject in mixed_subjects: 
                    subject_type = 'patient'
                    amblyo_type = 'mixed'
            elif dataset == 'control':
                subject_type = 'control'
                amblyo_type = 'control'
            
            # Define pRF eccenticity category (foveal or peripheral)
            def categorize_ecc(value):
                if value <= foveal_bound: return 'foveal'
                else: return 'peripheral'
            data_th['ecc_category'] = data_th['prf_size'].apply(categorize_ecc)
    
            # Determine weighted average and std per subjects of pRF size for fovea and periphery
            def weighted_average(df_groupby, column_data, column_weight):
                return (df_groupby[column_data] * df_groupby[column_weight]).sum() / df_groupby[column_weight].sum()
    
            # Compute table of weighted average by 'roi' and 'ecc_category'
            df_prf_ecc_cat = data_th.groupby(['roi', 'ecc_category']).apply(weighted_average, 'prf_size', 'prf_loo_r2').reset_index(name='prf_size_weighted_avg')
            df_prf_ecc_cat['pcm_weighted_avg'] = data_th.groupby(['roi', 'ecc_category']).apply(weighted_average, 'pcm', 'prf_loo_r2').reset_index()[0]
            df_prf_ecc_cat['prf_loo_r2_weighted_avg'] = data_th.groupby(['roi', 'ecc_category']).apply(weighted_average, 'prf_loo_r2', 'prf_loo_r2').reset_index()[0]
            df_prf_ecc_cat['prf_size_weighted_avg'] = data_th.groupby(['roi', 'ecc_category']).apply(weighted_average, 'prf_size', 'prf_loo_r2').reset_index()[0]
            df_prf_ecc_cat['vert_area_sum'] = data_th.groupby(['roi', 'ecc_category'])['vert_area'].sum().reset_index()['vert_area']
            df_prf_ecc_cat['subject'] = subject
            df_prf_ecc_cat['subject_type'] = subject_type
            df_prf_ecc_cat['amblyo_type'] = amblyo_type

            # Save tsv
            tsv_prf_ecc_cat_fn = "{}/{}_prf_ecc_cat.tsv".format(tsv_dir, subject)
            print('Saving tsv ({}: {} {}): {}'.format(subject, subject_type, amblyo_type, tsv_prf_ecc_cat_fn))
            df_prf_ecc_cat.to_csv(tsv_prf_ecc_cat_fn, sep="\t", na_rep='NaN', index=False)

Format: fsnative
Dataset: amblyo
Saving tsv (sub-01: patient aniso): /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-01/fsnative/prf/tsv/sub-01_prf_ecc_cat.tsv
Saving tsv (sub-02: patient strab): /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-02/fsnative/prf/tsv/sub-02_prf_ecc_cat.tsv
Saving tsv (sub-03: patient strab): /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-03/fsnative/prf/tsv/sub-03_prf_ecc_cat.tsv
Saving tsv (sub-04: patient aniso): /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-04/fsnative/prf/tsv/sub-04_prf_ecc_cat.tsv
Saving tsv (sub-05: patient strab): /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-05/fsnative/prf/tsv/sub-05_prf_ecc_cat.tsv
Saving tsv (sub-06: patient mixed): /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-06/fsnative/prf/tsv/sub-06_prf_ecc_cat.tsv
Saving tsv (sub-07: patient aniso): /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/

In [26]:
#### Group analysis
# get data of all subjects and combined them
for format_ in formats:
    print('Format: {}'.format(format_))
    for j, dataset in enumerate(datasets):
        if dataset == 'amblyo':
            dataset_dir = amblyo_dir
            list_subjects = amblyo_subjects
        elif dataset == 'control':
            dataset_dir = control_dir
            list_subjects = control_subjects
        
        for i, subject in enumerate(list_subjects):
            
            # Load data thresholded
            tsv_dir = '{}/{}/derivatives/pp_data/{}/{}/prf/tsv'.format(
                data_dir, dataset_dir, subject, format_)
            tsv_prf_ecc_cat_fn = "{}/{}_prf_ecc_cat.tsv".format(tsv_dir, subject)
            df_prf_ecc_cat_indiv = pd.read_table(tsv_prf_ecc_cat_fn, sep="\t")

            print('Adding {} {}: {}'.format(dataset, subject, tsv_prf_ecc_cat_fn))
            if i == 0 and j==0 :df_prf_ecc_cat = df_prf_ecc_cat_indiv.copy()
            else: df_prf_ecc_cat = pd.concat([df_prf_ecc_cat, df_prf_ecc_cat_indiv])

    # Subject type: control vs. patient
    df_prf_ecc_cat_subject_type_mean = df_prf_ecc_cat.groupby(['roi', 'ecc_category', 'subject_type', ]).mean()
    df_prf_ecc_cat_subject_type_mean.columns = ['prf_size_weighted_avg_mean', 'pcm_weighted_avg_mean', 'prf_loo_r2_weighted_avg_mean', 'vert_area_sum_mean']
    df_prf_ecc_cat_subject_type_sem = df_prf_ecc_cat.groupby(['roi', 'ecc_category', 'subject_type', ]).sem()
    df_prf_ecc_cat_subject_type_sem.columns = ['prf_size_weighted_avg_sem', 'pcm_weighted_avg_sem', 'prf_loo_r2_weighted_avg_sem', 'vert_area_sum_sem']
    df_prf_ecc_cat_subject_type = pd.concat([df_prf_ecc_cat_subject_type_mean, df_prf_ecc_cat_subject_type_mean], axis=1)

    # Amblyo type: n/a vs. aniso vs. strab vs. mixed
    df_prf_ecc_cat_amblyo_type_mean = df_prf_ecc_cat.groupby(['roi', 'ecc_category', 'amblyo_type', ]).mean()
    df_prf_ecc_cat_amblyo_type_mean.columns = ['prf_size_weighted_avg_mean', 'pcm_weighted_avg_mean', 'prf_loo_r2_weighted_avg_mean', 'vert_area_sum_mean']
    df_prf_ecc_cat_amblyo_type_sem = df_prf_ecc_cat.groupby(['roi', 'ecc_category', 'amblyo_type', ]).sem()
    df_prf_ecc_cat_amblyo_type_sem.columns = ['prf_size_weighted_avg_sem', 'pcm_weighted_avg_sem', 'prf_loo_r2_weighted_avg_sem', 'vert_area_sum_sem']
    df_prf_ecc_cat_amblyo_type = pd.concat([df_prf_ecc_cat_amblyo_type_mean, df_prf_ecc_cat_amblyo_type_sem], axis=1)

    # Difference by subject_type
    # save per format in amblyoprf

Format: fsnative
Adding amblyo sub-01: /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-01/fsnative/prf/tsv/sub-01_prf_ecc_cat.tsv
Adding amblyo sub-02: /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-02/fsnative/prf/tsv/sub-02_prf_ecc_cat.tsv
Adding amblyo sub-03: /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-03/fsnative/prf/tsv/sub-03_prf_ecc_cat.tsv
Adding amblyo sub-04: /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-04/fsnative/prf/tsv/sub-04_prf_ecc_cat.tsv
Adding amblyo sub-05: /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-05/fsnative/prf/tsv/sub-05_prf_ecc_cat.tsv
Adding amblyo sub-06: /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-06/fsnative/prf/tsv/sub-06_prf_ecc_cat.tsv
Adding amblyo sub-07: /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-07/fsnative/prf/tsv/sub-07_prf_ecc_cat.tsv
Adding amblyo sub-08: /home/mszinte/disks/meso_S/data/ambly

NameError: name 'ddd' is not defined

In [61]:
df_prf_ecc_cat_amblyo_type.loc[('V1', 'foveal')]

,prf_size_weighted_avg_mean,pcm_weighted_avg_mean,prf_loo_r2_weighted_avg_mean,vert_area_sum_mean,prf_size_weighted_avg_sem,pcm_weighted_avg_sem,prf_loo_r2_weighted_avg_sem,vert_area_sum_sem
amblyo_type,,,,,,,,
aniso,1.265954,3.027512,0.224644,999.362063,0.167565,0.851214,0.042881,394.003750
control,0.973166,10.362729,0.456440,2409.021604,0.025658,0.440930,0.012356,100.045796
mixed,1.018589,2.972015,0.185486,954.127697,0.127382,0.642308,0.035987,235.354895
strab,1.130263,3.218183,0.192330,1210.885901,0.144845,0.709619,0.038227,329.656613


In [30]:
df_prf_ecc_cat_amblyo_type_sem

prf_size_weighted_avg_sem  \
roi  ecc_category amblyo_type                              
LO   foveal       aniso                         0.122050   
                  control                       0.030800   
                  mixed                         0.181098   
                  strab                         0.362070   
     peripheral   aniso                         0.332380   
...                                                  ...   
sIPS peripheral   control                       0.365125   
                  mixed                         0.836017   
                  strab                         1.027372   
sPCS foveal       control                       0.073857   
     peripheral   control                       0.398795   

                               pcm_weighted_avg_sem  \
roi  ecc_category amblyo_type                         
LO   foveal       aniso                    0.712302   
                  control                  0.560049   
                  mixed                    0.674838   
                  strab                    0.402362   
     peripheral   aniso                    0.646864   
...                                             ...   
sIPS peripheral   control                  0.301640   
                  mixed                    0.031240   
                  strab                    0.111786   
sPCS foveal       control                  0.196193   
     peripheral   control                  0.197473   

                               prf_loo_r2_weighted_avg_sem  vert_area_sum_sem  
roi  ecc_category amblyo_type                                                  
LO   foveal       aniso                           0.019438          84.442676  
                  control                         0.012415          50.452661  
                  mixed                           0.027212          91.376596  
                  strab                           0.015668          99.237474  
     peripheral   aniso                           0.028932         128.932187  
...                                                    ...                ...  
sIPS peripheral   control                         0.012267         209.402352  
                  mixed                           0.005677          49.737519  
                  strab                           0.010021         128.439332  
sPCS foveal       control                         0.005582          30.408996  
     peripheral   control                         0.013897         117.835863  

[78 rows x 4 columns]